In [1]:
#################################
# Novel ML Attendance Taker
# Andrew Doyle
# 2/15/2025
# Using Yolo12 from ultralytics to detect items for class attendance while allowing a margin of tolerance due to probabilities of image counts being the same.
# Swapped to a segmented model as it appears to 
# Model Source:
# https://docs.ultralytics.com/tasks/segment/
#################################
import numpy as np
import math
import os
import cv2
import time
from ultralytics import YOLO
import pandas as pd
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from datetime import datetime
import yaml
###############
# POTENTIAL ITEMS AND THEIR RESPECTIVE KEYS
#names_list={0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 
#'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 
#26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 
#38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 
#52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 
#66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}
#################
# Important Lib Versions
# Numpy 1.26.4
# ultralytics 8.3.78
# python 3.12.4
# pandas 2.2.2
# opencv 4.11.2
#################
PYTHON_WD = os.getcwd()
SIZE = [4,6] # Rows, Columns
IMAGE_SIZE = [200,200] # Tile dimensions on collage
ITEMS_LIST = ['dog','airplane','bus', 'stop sign','cat','mouse'] # Populate with items in image library for script 
TOL = 0.20 # Image error tolerance
PERSON_THRESHOLD= 0.125 # Portion of image a person bounding box must occupy
#################

class ML_Attendance:
    
    def __init__(self, size, image_size, items_list, tol, person_threshold, model_path): # Set Parameters
        self.size =size
        self.image_size = image_size
        self.items_list = items_list
        tic = time.time()
        os.chdir(model_path)
        self.model = YOLO("yolo12x.pt")#.to('cpu')  # pretrained YOLO model
        toc = time.time()
        runtime = toc-tic
        print(f'Detector took: {runtime:.3f} to init')
        self.tol = tol
        self.person_threshold = person_threshold

    def _create_img_array_(self,seed):
        np.random.seed(seed)
        self.rand_arr=np.random.randint(len(ITEMS_LIST), size=SIZE)
        unique_items=np.unique(self.rand_arr)
        temp_count = {u: 0 for u in unique_items}
        for key in unique_items:
            for item in self.rand_arr.flatten():
                if item == key:
                    temp_count[key] += 1
        self.exp_item_count = {self.items_list[i]: temp_count[old_key] for i, old_key in enumerate(temp_count)}
        self.unused_items = [item for item in self.items_list if item not in self.exp_item_count or self.exp_item_count[item] == 0]
        self.used_items = [self.items_list[i] for i in unique_items]

    def _item_stitch_(self):
        x=0
        stitched_list=[['' for i in range(self.size[1])] for j in range(self.size[0])]
        for sub_list in self.rand_arr:
            y=0
            for item in sub_list:
                stitched_list[x][y]=f'{self.items_list[item]}.jpg'
                y+=1
            x+=1
        self.image_paths = stitched_list
    
    def _create_collage_(self,output_name):
        collage = np.zeros((self.size[0]*self.image_size[1], self.size[1]*self.image_size[0],3),dtype=np.uint8)
        for row_index in range(self.size[0]):
            for col_index in range(self.size[1]):
                img=cv2.imread(self.image_paths[row_index][col_index])
                img = cv2.resize(img, self.image_size)
                collage[row_index * self.image_size[1]:(row_index + 1) * self.image_size[1], col_index * self.image_size[0]:(col_index + 1) * self.image_size[0], :] = img
        return collage

    def _save_collage_(self,collage,output_name):
        df = pd.DataFrame({'rand_arr':[self.rand_arr],'exp_item_count':[self.exp_item_count],'unused_items':[self.unused_items],'used_items':[self.used_items]})
        df.to_json(f'{output_name}.json')
        cv2.imwrite(f'{output_name}.jpg',collage)

    def _detect_obj_(self):
        self.detections = []
        results = self.model.predict(self.img, imgsz = 1280)
        boxes = results[0].boxes
        t_label= boxes.cpu().numpy().cls.flatten()
        names_list={0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}
        label = [names_list[i] for i in t_label]
        bbox = boxes.cpu().numpy().xyxy.round()
        for i in range(boxes.shape[0]):
            self.detections.append({'name':label[i],'box_points':bbox[i]})
        
    
    def _obj_counter_(self):
        obj_count = {u: 0 for u in self.used_items}
        obj_count.update({'miss':0})
        obj_count.update({'unused':0})
        for i, item in enumerate(self.used_items):
            for j, detection in enumerate(self.detections):
                if detection['name'] == item:
                    obj_count[item] += 1
                elif (detection['name'] in self.unused_items) and detection['name'] != 'person': 
                    obj_count['unused'] += 1
        for item2 in self.exp_item_count.keys():
            obj_count['miss'] += np.abs(self.exp_item_count[item2]-obj_count[item2])
        self.obj_count = obj_count
        
    def _evaluate_counts_(self): # Finds the difference between expected count and found
        self.error = 0
        max_error = math.floor(self.size[0]*self.size[1]*self.tol)
        print(f'Max Error: {max_error}')
        for key in self.exp_item_count.keys():
            if self.obj_count[key] != self.exp_item_count[key]:
                self.error += np.abs(self.obj_count[key]-self.exp_item_count[key])
        if self.error> max_error:
            self.suff_match = False
        else:
            self.suff_match = True
            
    def _check_person_(self):
        self.person_flag = False
        for detection in self.detections:
            if detection['name'] == 'person':
                x=detection['box_points'][2] - detection['box_points'][0]
                y=detection['box_points'][3] - detection['box_points'][1]
                person_area = x*y
                if person_area >= self.area * self.person_threshold:
                    self.person_flag = True
                    print(f'Person found with percentage: {person_area/self.area * 100:.2f}%')
                    return
                else:
                    self.person_flag = False
                    
    def create_code_img(self,img_lib,output_path,output_name,seed=np.random.randint(time.time())): # Executes all functions to properly create the collage
        tic = time.time()
        self._create_img_array_(seed)
        self._item_stitch_()
        current_directory = os.getcwd()
        os.chdir(img_lib)
        collage = self._create_collage_(output_name)
        os.chdir(output_path)
        self._save_collage_(collage,output_name)
        os.chdir(current_directory)
        toc = time.time()
        runtime = toc-tic
        print(f'Image Generation time : {runtime:.3f}')
        
    def read_data(self, output_path, filename):
        os.chdir(output_path)
        df = pd.read_json(filename)
        self.rand_arr = df.rand_arr.to_list()[0]
        self.exp_item_count = df.exp_item_count.to_dict()[0]
        self.unused_items = df.unused_items.to_list()[0]
        self.used_items = df.used_items.to_list()[0]

    def evaluate_img(self, img): # Executes all functions to evaluate an image and compare datapoints to the source.
        tic1 = time.time()
        self.img= img
        self.height, self.width, channels = cv2.imread(img).shape
        self.area = self.height*self.width
        tic2= time.time()
        self._detect_obj_()
        toc2 = time.time()
        run_time2 = toc2-tic2
        print(f'Yolo12x Runtime: {run_time2:.3f}')
        self._obj_counter_()
        self._evaluate_counts_()
        self._check_person_()
        toc1 = time.time()
        run_time1 = toc1-tic1
        print(f'Evaluation Runtime: {run_time1:.3f}')
        # print(f'Total Error: {self.error}\nSufficient Match?: {self.suff_match}\n{self.person_threshold*100:.2f}% Person? {self.person_flag}\nUnused Items in photo: {self.obj_count['unused']}')
        if not self.person_flag or not self.suff_match:
            print(f'Reccommending review for Image: {self.img}')
        return self.error, self.suff_match, self.person_flag, self.suff_match and self.person_flag


    

In [2]:
Evaluater = None # Predefine var useful later

####################################
# CONFIG Functions
####################################
def load_config(config=os.path.join(PYTHON_WD,'config.yml')):
    print(config)
    if os.path.exists(config):
        with open(config,'r') as f:
            config_data = yaml.safe_load(f)
            mod_path = config_data['mod_path']
            out_path = config_data['out_path']
            lib_path = config_data['lib_path']
            input_path = config_data['input_path']
            tol = config_data['tol']
            person_threshold = config_data['person_threshold']
        return mod_path,out_path,lib_path,input_path,tol,person_threshold
    else:
        print('Config Not Found')
        return '','','','',0.20,0.125

def save_config(mod_path,out_path,lib_path,input_path,tol,person_threshold):
    with open(os.path.join(PYTHON_WD,'config.yml'),'w') as f:
        config_data = {'mod_path':mod_path,'out_path':out_path,'lib_path':lib_path,'input_path':input_path,'tol':tol,'person_threshold':person_threshold}
        yaml.dump(config_data, f)
    return

def file_sel_gui():
    f=tk.filedialog.askdirectory()
    return f

####################################
# GUI Button Functions
####################################
def validate_float_input(P):
    # Check if the input is a valid float (allowing empty string for backspace).
    if P == "" or P.replace('.', '', 1).isdigit():
        try:
            float(P)  # Try to convert the input to a float.
            return True
        except ValueError:
            return False
    return False


def generate_img_gui(img_lib,output_path,model_path):
    Evaluater = ML_Attendance(SIZE, IMAGE_SIZE, ITEMS_LIST, TOL, PERSON_THRESHOLD,model_path)
    sec = time.time()
    timestamp = datetime.fromtimestamp(sec).strftime('%Y-%m-%d-%H-%M-%S')
    fname = tk.simpledialog.askstring('Image Dialog','Image Name: ', initialvalue=timestamp)
    Evaluater.create_code_img(img_lib,output_path,fname)
    
def eval_bulk(input_path,model_path):
    Evaluater = ML_Attendance(SIZE, IMAGE_SIZE, ITEMS_LIST, TOL, PERSON_THRESHOLD,model_path)

    for filename in os.listdir(file_path):
    # Construct the full file path
        full_path = os.path.join(file_path, filename)

        # Check if it is a file and an image (you may need to refine the extensions)
        if os.path.isfile(full_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp','.heic','.tif','.tiff')):
            # Process the image file
            print(f"Processing image: {full_path}")
            
            Evaluater.evaluate_img(cv2.imread(full_path))
            # Add your image processing logic here, e.g., opening with PIL or other libraries
        elif os.path.isfile(full_path):
          print(f"Skipping non-image file: {full_path}")
##########################
# GUI Stuff
##########################
root = tk.Tk()

fpath_model = tk.StringVar()
fpath_output = tk.StringVar()
fpath_imglib = tk.StringVar()
fpath_config = tk.StringVar()
fpath_input = tk.StringVar()
gui_tol = tk.DoubleVar()
gui_person_threshold = tk.DoubleVar()


temp = load_config()

fpath_model.set(temp[0])
fpath_output.set(temp[1])
fpath_imglib.set(temp[2])
fpath_input.set(temp[3])
gui_tol.set(temp[4])
gui_person_threshold.set(temp[5])

del temp
root.title('ML Attendance')
frm = ttk.Frame(root,padding = 10)
frm.grid()

ttk.Label(frm, text='Model Path').grid(column=0,row=0)
model_entry=tk.Entry(frm,textvariable=fpath_model,width = 50)
model_entry.grid(column=0,row=1,columnspan=5)
ttk.Button(frm,text='File Select',command = lambda: fpath_model.set(file_sel_gui())).grid(column=6, row=1,sticky='w')

ttk.Label(frm, text='Output Path').grid(column=0,row=2)
output_entry=tk.Entry(frm,textvariable=fpath_output,width = 50)
output_entry.grid(column=0,row=3,columnspan=5)
ttk.Button(frm,text='File Select',command = lambda: fpath_output.set(file_sel_gui())).grid(column=6, row=3,sticky='w')

ttk.Label(frm, text='Img Lib Path').grid(column=0,row=4)
imglib_entry=tk.Entry(frm,textvariable=fpath_imglib,width = 50)
imglib_entry.grid(column=0,row=5,columnspan=5)
ttk.Button(frm,text='File Select',command = lambda: fpath_imglib.set(file_sel_gui())).grid(column=6, row=5,sticky='w')

ttk.Label(frm, text='Input Folder Path').grid(column=0,row=6)
input_entry=tk.Entry(frm,textvariable=fpath_input,width = 50)
input_entry.grid(column=0,row=7,columnspan=5)
ttk.Button(frm,text='File Select',command = lambda: fpath_input.set(file_sel_gui())).grid(column=6, row=7,sticky='w')

ttk.Label(frm,text = 'Note: Config Generates on quit into Python file\'s Directory').grid(column=0,row=8,columnspan=5)

ttk.Label(frm,text='Error Tolerance:').grid(column=0,row=9)
tol_entry=tk.Entry(frm, validate='key', textvariable=gui_tol,validatecommand=(root.register(validate_float_input), '%P')).grid(column=1,row=9)
ttk.Label(frm,text='Person Thresh:').grid(column=2,row=9)
person_thresh_entry=tk.Entry(frm, validate='key', textvariable=gui_person_threshold,validatecommand=(root.register(validate_float_input), '%P')).grid(column=3,row=9)


ttk.Button(frm,text='Generate Img',command = lambda: generate_img_gui(fpath_imglib.get(),fpath_output.get(),fpath_model.get())).grid(column=1, row=10,sticky='w')
ttk.Button(frm,text='Evaluate',command = lambda:eval_bulk()).grid(column=2, row=10,sticky='w')
ttk.Button(frm,text='Save & Quit',command = lambda:[save_config(fpath_model.get(),fpath_output.get(),fpath_imglib.get(),fpath_input.get(),gui_tol.get(),gui_person_threshold.get()),root.destroy()]).grid(column=3, row=10,sticky='w')
root.mainloop()




C:\Users\andre\Jupyter\ML_Attendance\ml_attendance\config.yml
Config Not Found
